In [14]:
import os
import re
import json
import pandas as pd
from src.utils import read_config
from prompts import gemini_prompts
import google.generativeai as genai
import google.ai.generativelanguage as glm

from langchain_huggingface import HuggingFaceEmbeddings
from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.faiss import FAISS, DistanceStrategy
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)

from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever


import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## 0609 notes
1. Add KG(knowledge graph) as part of metadata
2. Use SelfQueryRetriver to get the top_k answer
3. Override the `_get_docs_with_query` to use the `similarity_search_with_score` method(or with `similarity_search_with_score_by_vector`)

In [15]:
configs = read_config('.env/configs.json')
api_key =configs['g_key']
os.environ["GOOGLE_API_KEY"] = api_key
genai.configure(api_key=api_key)

In [16]:
model_name = "sentence-transformers/distiluse-base-multilingual-cased-v1"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

c:\Users\sean.chang\AppData\Local\anaconda3\envs\project_envelope\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [17]:
# vector_db_path = "./vector_db_2/"

# # load FAISS
# vectorstore = FAISS.load_local(folder_path=vector_db_path, allow_dangerous_deserialization=True, embeddings=embeddings)

In [32]:
# chromadb
vector_db_path = "./chroma_db/"
chroma_collection_name = "correction_v1"
vectorstore = Chroma(persist_directory=vector_db_path, collection_name=chroma_collection_name, embedding_function=embeddings)

In [33]:
vectorstore._collection.count()

75

### Function for getting triple

In [20]:
llm = ChatGoogleGenerativeAI(
    model="gemini-pro",
    convert_system_message_to_human=True,
    safety_settings={
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    },
)

In [21]:
metadata_field_info = [
    AttributeInfo(
        name='source',
        description='the name of the document',
        type='string'
    ),
    AttributeInfo(
        name='target',
        description='the receiver of the document',
        type='string',
    ),
    AttributeInfo(
        name='reason',
        description='the reason of having this document',
        type='string',
    ),
    AttributeInfo(
        name='fact',
        description='the fact of the document',
        type='string',
    ),
    AttributeInfo(
        name='keywords',
        description='the keywords of the reason part of the document',
        type='string'
    ),
    AttributeInfo(
        name='relationship_between_entities',
        description='the relations between entities in the document',
        type='string'
    )
]

In [22]:
top_k = 3
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_contents="test",
    metadata_field_info=metadata_field_info,
    k=top_k,
    search_type='similarity',
    search_kwagrs={"k":top_k},
    enable_limit=True
)

# additional query
# mixed_search_string = additional_query + "與" + query

In [23]:
query = "行政院原住民"
retriever.get_relevant_documents(query)

c:\Users\sean.chang\AppData\Local\anaconda3\envs\project_envelope\lib\site-packages\langchain_google_genai\chat_models.py:344: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


[Document(page_content='行政院客家委員會、行政院原住民族委員會及僑務委員會將客家電視、原住民電視及臺灣宏觀電視等頻道節目之製播，依政府採購法委由財團法人公共電視文化事業基金會辦理，雖非無據，惟造成公視基金會與客委會、原民會及僑委會針對上開頻道節目之製播各有立場、互生歧見，致紛爭不斷，詎行政院新聞局坐視上開爭端，非但未積極研修法令，亦未尋求妥適解決辦法，核有怠失，爰依法提案糾正', metadata={'fact': '行政院客家委員會、行政院原住民族委員會及僑務委員會(下稱客委會、原民會及僑委會)將客家電視、原住民電視及臺灣宏觀電視(下稱客視、原視及宏觀電視)等頻道節目之製播，依政府採購法委由財團法人公共電視文化事業基金會(下稱公視基金會)辦理，雖非無據，惟造成公視基金會與客委會、原民會及僑委會針對上開頻道節目之製播各有立場、互生歧見，致紛爭不斷，詎行政院新聞局(下稱新聞局)坐視上開爭端，非但未積極研修法令，亦未尋求妥適解決辦法，核有怠失，茲提出事實與理由於后一、據陳訴人指陳略以，95年無線公股處理條例於立法院三讀通過，目的在使黨、政、軍退出媒體，95年7月17日行政院召開「研商『無電視事業公股處理條例』規範原民台、客家台及宏觀頻道交由公共電視台製播2相關機關間溝通協調機制之建立問題」會議，會中決議「其製播所需經費之移撥(無政府採購法之適用問題)…」詎客委會、原民會及僑委會未依該決議辦理，仍依政府採購法，以標案形式由公視基金會辦理，實非合理又公視基金會辦理「原、客、宏觀頻道之計畫與製播」，並無獨立之經費得以辦理，而係由政府部門辦理審查、驗收，始得請領款項之情況，違反無線公股處理條例黨政軍退出媒體之精神，造成公共電視新聞自主性、專業性之箝制，詎該公股處理條例對「經費」疏漏等問題，迄未立法補正，亦造成公共電視台(下稱公視)財務經營之困境目前公視代為製播「原、客、宏觀頻道」，亦遭客委會、原民會及僑委會等延宕付款，更造成公視財務經營困難等語云云二、依據「無線電視事業公股處理條例」第14條第3項所定「政府編列預算招標採購或設置之客家電視、原住民電視、台灣宏觀電視等頻道節目之製播，應於本條例公布施行後之次年度起，『交由』公視基金會辦理」目前公視基金會係依照政府採購法之規定，定期向客委會、原民會及僑委會提出營運計畫書、工作報告、驗收作業等相關作業惟公視基金

In [24]:
from typing import Any, Dict


class CustomSelfQueryRetriever(SelfQueryRetriever):
    def _get_docs_with_query(
        self, query: str, search_kwargs: Dict[str, Any]
    ):
        """Get docs, adding score information."""
        docs, scores = zip(
            *vectorstore.similarity_search_with_score(query, **search_kwargs)
        )
        for doc, score in zip(docs, scores):
            doc.metadata["score"] = score

        return docs

In [25]:
document_content_description = 'agent'
retriever = CustomSelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
)


retriever.invoke("行政院原住民")


c:\Users\sean.chang\AppData\Local\anaconda3\envs\project_envelope\lib\site-packages\langchain_google_genai\chat_models.py:344: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


(Document(page_content='行政院客家委員會、行政院原住民族委員會及僑務委員會將客家電視、原住民電視及臺灣宏觀電視等頻道節目之製播，依政府採購法委由財團法人公共電視文化事業基金會辦理，雖非無據，惟造成公視基金會與客委會、原民會及僑委會針對上開頻道節目之製播各有立場、互生歧見，致紛爭不斷，詎行政院新聞局坐視上開爭端，非但未積極研修法令，亦未尋求妥適解決辦法，核有怠失，爰依法提案糾正', metadata={'fact': '行政院客家委員會、行政院原住民族委員會及僑務委員會(下稱客委會、原民會及僑委會)將客家電視、原住民電視及臺灣宏觀電視(下稱客視、原視及宏觀電視)等頻道節目之製播，依政府採購法委由財團法人公共電視文化事業基金會(下稱公視基金會)辦理，雖非無據，惟造成公視基金會與客委會、原民會及僑委會針對上開頻道節目之製播各有立場、互生歧見，致紛爭不斷，詎行政院新聞局(下稱新聞局)坐視上開爭端，非但未積極研修法令，亦未尋求妥適解決辦法，核有怠失，茲提出事實與理由於后一、據陳訴人指陳略以，95年無線公股處理條例於立法院三讀通過，目的在使黨、政、軍退出媒體，95年7月17日行政院召開「研商『無電視事業公股處理條例』規範原民台、客家台及宏觀頻道交由公共電視台製播2相關機關間溝通協調機制之建立問題」會議，會中決議「其製播所需經費之移撥(無政府採購法之適用問題)…」詎客委會、原民會及僑委會未依該決議辦理，仍依政府採購法，以標案形式由公視基金會辦理，實非合理又公視基金會辦理「原、客、宏觀頻道之計畫與製播」，並無獨立之經費得以辦理，而係由政府部門辦理審查、驗收，始得請領款項之情況，違反無線公股處理條例黨政軍退出媒體之精神，造成公共電視新聞自主性、專業性之箝制，詎該公股處理條例對「經費」疏漏等問題，迄未立法補正，亦造成公共電視台(下稱公視)財務經營之困境目前公視代為製播「原、客、宏觀頻道」，亦遭客委會、原民會及僑委會等延宕付款，更造成公視財務經營困難等語云云二、依據「無線電視事業公股處理條例」第14條第3項所定「政府編列預算招標採購或設置之客家電視、原住民電視、台灣宏觀電視等頻道節目之製播，應於本條例公布施行後之次年度起，『交由』公視基金會辦理」目前公視基金會係依照政府採購法之規定，定期向客委會、原民會及僑委會提出營運計畫書、工作報告、驗收作業等相關作業惟公視基金

In [26]:
CONCLUSION_PROMPT="""
### INSTRUCTION: 說明查詢的內容 QUERY 與 REF 中的之間的關係。若找不到合理的關係則說沒有明確的相關性，不要亂說
### QUERY: {query}
### REF: {reference}
### ASSISTANT: 
"""

In [27]:
# response_data = {}
# for i, page in enumerate(result):
#     print(page.metadata['source'])
#     # summary_prompt = PromptTemplate.from_template(gemini_prompts.SUMMARY_PROMPT)
#     # chain = summary_prompt | llm
#     # summary_data = {
#     #     "reference": page.page_content
#     # }

#     # summary_response = chain.invoke(summary_data)
    
#     # pattern = r'\*\*摘要：\*\*'
#     # if re.search(pattern, summary_response.content):
#     #     re.sub(pattern, '', summary_response.content)
        
#     suffix = "### ASSISTANT: "
#     input_variables = ["query", "reference"]

#     conclusion_prompt = PromptTemplate.from_examples(CONCLUSION_PROMPT, suffix, input_variables)
#     chain = conclusion_prompt | llm
#     conclusion_data = {
#         "query": query,
#         "reference": page.page_content
#     }
#     conclusion_response = chain.invoke(conclusion_data)

#     # response_data[i] = {
#     #     "summary": summary_response.content,
#     #     "source": page.metadata['source'],
#     #     "score": f"{page.metadata['score']:.2f}",
#     #     "target": page.metadata['target'],
#     #     # "conclusion": conclusion_response.content
#     # }
    



In [28]:
metadata_field_info = [

    AttributeInfo(
        name='target',
        description='受文對象、受文者',
        type='string',
    ),
    AttributeInfo(
        name='reason',
        description='被糾正原因',
        type='string',
    ),
    AttributeInfo(
        name='fact',
        description='被糾正的證據',
        type='string',
    ),
    AttributeInfo(
        name='keywords',
        description='被糾正原因中的關鍵字',
        type='string'
    ),
    AttributeInfo(
        name='relationship_between_entities',
        description='被糾正原因中呈現三元組關係的entities(實體)',
        type='string'
    )
]

In [38]:
query = '受文者是中華郵政'

In [39]:
class CustomSelfQueryRetriever(SelfQueryRetriever):
        def _get_docs_with_query(
            self, query: str, search_kwargs: Dict[str, Any]
        ):
            """Get docs, adding score information."""
            docs, scores = zip(
                *vectorstore.similarity_search_with_score(query, **search_kwargs)
            )
            for doc, score in zip(docs, scores):
                if score < 1:
                    doc.metadata["score"] = 1-score
                elif score >1 :
                    doc.metadata["score"] = 1

            return docs
        
document_content_description = 'agent'
retriever = CustomSelfQueryRetriever.from_llm(
        llm,
        vectorstore,
        document_content_description,
        metadata_field_info,
        verbose=True
)

retriever.get_relevant_documents(query)


c:\Users\sean.chang\AppData\Local\anaconda3\envs\project_envelope\lib\site-packages\langchain_google_genai\chat_models.py:344: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


ValueError: not enough values to unpack (expected 2, got 0)

In [31]:
for d in test_docs:
    print(d.metadata['target'])

行政院新聞局


In [58]:
class CustomSelfQueryRetriever(SelfQueryRetriever):
        def _get_docs_with_query(
            self, query: str, search_kwargs: Dict[str, Any]
        ):
            """Get docs, adding score information."""
            docs, scores = zip(
                *vectorstore.similarity_search_with_score(query, **search_kwargs)
            )
            for doc, score in zip(docs, scores):
                if score < 1:
                    doc.metadata["score"] = 1-score
                elif score >1 :
                    doc.metadata["score"] = 1

            return docs
        
document_content_description = 'agent'
retriever = CustomSelfQueryRetriever.from_llm(
        llm,
        vectorstore,
        document_content_description,
        metadata_field_info,
        verbose=True
)
docs = retriever.invoke(query)
print(query)
response_data = {}
for i, page in enumerate(docs):
    print(page.metadata['source'])
    summary_prompt = PromptTemplate.from_template(gemini_prompts.SUMMARY_PROMPT)
    chain = summary_prompt | llm
    summary_data = {
        "reference": page.page_content
    }

    summary_response = chain.invoke(summary_data)
    
    pattern = r'\*\*摘要：\*\*'
    if re.search(pattern, summary_response.content):
        re.sub(pattern, '', summary_response.content)
        
        
    # conclusion_prompt = PromptTemplate.from_examples(gemini_prompts.CONCLUSION_PROMPT)
    # chain = conclusion_prompt | llm
    # conclusion_data = {
    #     "query" : query,
    #     "reference": page.page_content
    # }
    # conclusion_response = chain.invoke(conclusion_data)



    response_data[i] = {
        "summary": summary_response.content,
        "source": page.metadata['source'],
        "score": f"{page.metadata['score']:.2f}",
        "target": page.metadata['target'],
        # "conclusion": conclusion_response.content
    }

c:\Users\sean.chang\AppData\Local\anaconda3\envs\project_envelope\lib\site-packages\langchain_google_genai\chat_models.py:344: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


受文者包含行政院新聞局
52587.pdf


c:\Users\sean.chang\AppData\Local\anaconda3\envs\project_envelope\lib\site-packages\langchain_google_genai\chat_models.py:344: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


In [57]:
response_data

{0: {'summary': '行政院客委會、原民會、僑委會委託公視基金會製作客家電視、原住民電視和台灣宏觀電視的節目，但各單位對於節目的製作有不同立場，造成紛爭。新聞局未積極修法或尋求解決辦法，因此監察院提案糾正。',
  'source': '52587.pdf',
  'score': '1.01',
  'target': '行政院新聞局'}}

In [49]:
page.metadata

{'fact': '民國(下同)103年6月27日，國防部陸軍司令部後勤指揮部(下稱陸勤部)湳湖營區，疑未依程序銷毀逾期彈藥，致發生爆炸意外，3名承包商作業人員未著防護裝備，遭嚴重灼傷，其中1人傷重不治，爰經本院國防及情報委員會決議推派調查案經本院向國防部、陸勤部、行政院公共工程委員會（下稱行政院工程會）、勞動部、新竹縣政府暨所屬消防局等機關調閱有關卷證資料、法令詳於審閱；並針對本案調卷及案情相關疑點，請相關單位主管及承辦人員說明，並提供本案廢彈處理合約暨履約相關內容等全卷及職災相關檢查報告供參；復於104年2月4日詢問國防部後勤參謀次長室次長尚○○中將、助理次長崔○○少將、陸軍司令部參謀長郝○○中將、彈藥處處長簡○○少將、軍備局獲得管理處處長歐陽○○少將及相關承辦人等到院說明，該部嗣又先後於同年2月13日、3月18日及5月11日檢陳詢問補充說明及本案相關起訴與後續查證說明等資料到院，業調查竣事，茲臚列調查2發現之違失如次一、陸勤部本案未依政府採購法履約管理採購契約相關法令規定切實辦理，核有違失；且衡其對本案廢彈委商處理履約督管機制未臻健全及完善，致衍生本案爆炸傷亡重大職業災害悲劇憾事，自屬難辭其咎，亦核有明顯缺失，以上均核有違失，應切實檢討改進(一)經查，有關本案委商處理履約督管乙節，按行政院工程會所訂頒政府採購法第四章「履約管理」第63條規定「各類採購契約以採用主管機關訂定之範本為原則，其要項及內容由主管機關參考國際及國內慣例定之」同法第70條規定「機關辦理工程採購，應明訂廠商執行品質管理、環境保護、施工安全衛生之責任，並對重點項目訂定檢查程序及檢驗標準（第1項）機關於廠商履約過程，得辦理分段查驗，其結果並得供驗收之用（第2項）…財物或勞務採購需經一定履約過程，而非以現成財物或勞務供應者，準用第1項及第2項之規定（第5項）」另依該會所訂頒「公共工程施工品質管理作業要點」第15點規定「機關應隨時督導工程施工情形，並留存紀錄備查另得視工程需要設置工程督導小組，隨時進行施工品質督導工作機關發現工程缺失時，應即以書面通知監造單位或廠商限期改善」以上規定甚明(二)另依本案「勞動部重大職業災害檢查報告書」涉此內容摘錄該報告第十二項、「本災害構成其他法律罰則事項」-(三)關係事業單位（國家中山科學研究院系統製造中心）「該事業單位承攬國防部『國軍逾期傳統批號彈藥委商處理(契約編

In [84]:
query

'三元組中包含郵政法規定之郵件補償範疇'